# DSGVO-konforme Anonymisierung von Retail-Daten mit $k$-Anonymität, $l$-Diversität und $t$-Closeness
Dieses Notebook demonstriert die Schritte zur **DSGVO-konformen Anonymisierung** eines Retail-Datensatzes:
- **Entfernung direkter Identifikatoren** (z.B. Name, E-Mail).
- **Generalisierung von Quasi-Identifikatoren** (PLZ, Alter).
- **Überprüfung der Anonymisierungsgrade** anhand von $k$-Anonymität, $l$-Diversität und $t$-Closeness.

## Hintergrund: Schutz vor Re-Identifikation
Die verwendeten Methoden dienen dazu, das Risiko der **Re-Identifikation** von Personen zu minimieren, indem sichergestellt wird, dass jeder Datensatz in einer Äquivalenzklasse (Gruppe von Personen mit den gleichen Quasi-Identifikatoren) enthalten ist, die groß genug ist ($k$-Anonymität) und eine ausreichende Bandbreite an sensiblen Attributen aufweist ($l$-Diversität und $t$-Closeness).

## 💻 Code und Datenvorbereitung

In [1]:
# Installation der benötigten Bibliotheken
!pip install pandas scipy matplotlib

### Import benötigter Module

In [2]:
import pandas as pd
from scipy.spatial.distance import euclidean
import matplotlib.pyplot as plt

### Datensatzbeschreibung und -bezug

Der Datensatz stellt eine vereinfachte Retail-Datenbasis dar, in der Kundendaten wie Postleitzahl, Alter und Einkommen gespeichert sind. Diese Attribute werden genutzt, um Anonymisierungstechniken wie $k$-Anonymität, $l$-Diversität und $t$-Closeness zu demonstrieren. Dabei werden die **Quasi-Identifikatoren (PLZ, Alter)** so verallgemeinert, dass einzelne Personen nicht mehr eindeutig identifiziert werden können. Gleichzeitig bleibt die Vielfalt in sensiblen Attributen wie dem **Einkommen** gewährleistet, damit keine Rückschlüsse auf Einzelne möglich sind. Der Datensatz dient somit als Beispiel, um Methoden des datenschutzgerechten Umgangs mit personenbezogenen Daten praxisnah zu testen und zu evaluieren.

### 🔗 Link zum Datensatz

[Retail Transactional Dataset (Kaggle)](https://www.kaggle.com/datasets/bhavikjikadara/retail-transactional-dataset)

### Datensatz-Download (Kaggle API)

In [3]:
! pip install -q kaggle
! mkdir -p ~/.kaggle

**Wichtig:** Bitte die JSON-Zugangsdaten von Kaggle hier eingeben (im Format: `{"username":"....","key":"...."}`)

In [4]:
# Platzhalter: Tragen Sie hier Ihre JSON Cred als Dictionary ein
d_json_cred ={
    "username":"IHR_KAGGLE_USERNAME",
    "key":"IHR_KAGGLE_API_KEY"
}

Kaggle Zugangsdaten speichern und Berechtigungen setzen

In [5]:
import json
import os

# Sicherstellen, dass das .kaggle Verzeichnis existiert
os.makedirs("~/.kaggle", exist_ok=True)

# Schreibe die Zugangsdaten in die kaggle.json Datei
with open(os.path.expanduser("~/.kaggle/kaggle.json"), "w") as f:
    json.dump(d_json_cred, f)

# Berechtigung für die Konfigurationsdatei setzen
! chmod 600 ~/.kaggle/kaggle.json

In [6]:
# Datensatz herunterladen
!kaggle datasets download -d bhavikjikadara/retail-transactional-dataset

Unzip der Daten in ein lokales 'data'-Verzeichnis

In [7]:
!unzip -o retail-transactional-dataset.zip -d ./data

### Originaldaten einlesen und inspizieren

In [8]:
CSV_PATH = "./data/retail_data.csv"
df = pd.read_csv(CSV_PATH)

print("=== Originaldaten: Erste 5 Zeilen ===")
print(df.head(), "\n")
print("=== Spalten (Identifikatoren) ===")
print(df.columns.tolist())


### 🗑️ Direkte Identifikatoren entfernen und Quasi-Identifikatoren auswählen

**Direkte Identifikatoren** (wie Name, E-Mail, Telefon, Adresse, Customer_ID) werden entfernt. Für die Anonymisierung werden nur die folgenden Spalten beibehalten:

* **Quasi-Identifikatoren (QI)**: `Zipcode`, `Age`, `Country`
* **Sensibles Attribut (SA)**: `Income`

In [9]:
# Nur die gewünschten Spalten behalten (QI und SA)
keep_cols = ["Zipcode", "Age", "Country", "Income"]
df_reduced = df[keep_cols].copy() # .copy() um SettingWithCopyWarning zu vermeiden

# Neue CSV zur Weiterverarbeitung speichern (optional)
OUTPUT_PATH = "./data/retail_data_filtered.csv"
df_reduced.to_csv(OUTPUT_PATH, index=False)

print("Neue CSV gespeichert:", OUTPUT_PATH)
print("=== Gefilterte Daten (erste 5 Zeilen) ===")
print(df_reduced.head())


## 🛡️ Anonymisierung: Generalisierung und Überprüfung der Schutzkriterien

### Generalisierung der Quasi-Identifikatoren
Um die **$k$-Anonymität** zu ermöglichen, werden die Quasi-Identifikatoren `Zipcode` und `Age` generalisiert:
1.  **Postleitzahl (`Zipcode`)**: Auf die ersten **zwei Ziffern** reduziert, um die Region statt des exakten Ortes zu identifizieren.
2.  **Alter (`Age`)**: In drei breite **Intervalle** (`<=30`, `31-50`, `51+`) gruppiert.

In [10]:
import pandas as pd
from scipy.spatial.distance import euclidean

# 1. Gefilterte CSV laden
CSV_PATH = "./data/retail_data_filtered.csv"
df = pd.read_csv(CSV_PATH)

print("=== Originaldaten (gefiltert) ===")
print(df.head(), "\n")

# 2. Quasi-Identifikatoren generalisieren
# - PLZ auf die ersten 2 Stellen reduzieren
# - Alter in Intervalle gruppieren
df['Zipcode'] = df['Zipcode'].astype(str).str[:2]
df['Age'] = pd.cut(df['Age'], bins=[0, 30, 50, 100],
                   labels=['<=30', '31-50', '51+'], include_lowest=True)

# Parameter
k = 2
l = 2
t = 0.3
quasi_identifiers = ['Zipcode', 'Age', 'Country']  # Alle QIs verwenden
sensitive_attr = 'Income'  # sensibles Attribut
total_records = len(df)

print(f"Anzahl Datensätze Original: {total_records}")

### 3. $k$-Anonymität prüfen
# Gruppengröße (Äquivalenzklasse) berechnen
group_sizes = df.groupby(quasi_identifiers, observed=True).size().reset_index(name='Count')
# Nur Gruppen behalten, die mindestens k Datensätze haben
df_k_anonym = df.merge(group_sizes[group_sizes['Count'] >= k],
                       on=quasi_identifiers)

print(f"Nach k-Anonymität (k={k}): {len(df_k_anonym)} von {total_records} Datensätzen ({len(df_k_anonym)/total_records:.2%} erhalten)\n")

### 4. $l$-Diversität prüfen
# Prüffunktion: eine Gruppe ist l-divers, wenn sie mind. l verschiedene Werte des sensiblen Attributes enthält
def check_l_diversity(group):
    return group[sensitive_attr].nunique() >= l

# Nur Gruppen behalten, die l-divers sind
df_l_diverse = df_k_anonym.groupby(quasi_identifiers, observed=True).filter(check_l_diversity)
print(f"Nach l-Diversität (l={l}): {len(df_l_diverse)} von {len(df_k_anonym)} Datensätzen ({len(df_l_diverse)/len(df_k_anonym):.2%} erhalten)\n")

### 5. $t$-Closeness prüfen
# Gesamtverteilung des sensiblen Attributs
overall_dist = df[sensitive_attr].value_counts(normalize=True)

# Prüffunktion: eine Gruppe ist t-close, wenn ihre Verteilung nahe genug an der Gesamtverteilung liegt (gemessen mit Euklidischer Distanz)
def t_closeness(group):
    group_dist = group[sensitive_attr].value_counts(normalize=True)
    # gleiche Reihenfolge wie in overall_dist erzwingen, fehlende Werte = 0
    group_dist = group_dist.reindex(overall_dist.index, fill_value=0)
    # Euklidische Distanz zwischen Gruppen- und Gesamtverteilung berechnen
    distance = euclidean(overall_dist.values, group_dist.values)
    return distance <= t

# Nur Gruppen behalten, die t-close sind
df_t_close = df_l_diverse.groupby(quasi_identifiers, observed=True).filter(t_closeness)
print(f"Nach t-Closeness (t={t}): {len(df_t_close)} von {len(df_l_diverse)} Datensätzen ({len(df_t_close)/len(df_l_diverse):.2%} erhalten)\n")

# 6. Ergebnis anzeigen
# Die 'Count' Spalte wird beim Merge automatisch hinzugefügt
df_final = df_t_close.merge(group_sizes, on=quasi_identifiers)
print("=== Anonymisierte Daten (ersten 10 Zeilen und ihre Gruppengröße) ===")
print(df_final.head(10))


## 💡 Test der Anonymisierung mit höheren $k$-Werten (Idee)
Um die Auswirkung des Anonymisierungsgrads auf die verbleibende Datenmenge zu analysieren, ist es sinnvoll, verschiedene $k$-Werte zu testen. Je höher $k$ gewählt wird, desto stärker wird der Datenschutz, aber desto mehr Datensätze könnten verloren gehen. Dieser Abschnitt visualisiert diesen Trade-off.

In [11]:
def get_k_anonymized_count(df_input, quasi_identifiers, k_val):
    # Gruppengrößen berechnen
    group_sizes = df_input.groupby(quasi_identifiers, observed=True).size().reset_index(name='Count')
    # Filterung und Anzahl der verbleibenden Datensätze
    df_k_anonym = df_input.merge(group_sizes[group_sizes['Count'] >= k_val],
                                 on=quasi_identifiers)
    return len(df_k_anonym)

# Verwenden Sie die bereits generalisierten QIs (Daten müssen neu geladen werden, falls der vorherige Block nicht ausgeführt wurde)
CSV_PATH = "./data/retail_data_filtered.csv"
df_generalized = pd.read_csv(CSV_PATH)
df_generalized['Zipcode'] = df_generalized['Zipcode'].astype(str).str[:2]
df_generalized['Age'] = pd.cut(df_generalized['Age'], bins=[0, 30, 50, 100],
                   labels=['<=30', '31-50', '51+'], include_lowest=True)
quasi_identifiers = ['Zipcode', 'Age', 'Country']

# Testen von k-Werten
k_values = [2, 3, 5, 10, 20]
remaining_counts = []
total_records = len(df_generalized)

print(f"Anzahl Datensätze Original: {total_records}")

for k_val in k_values:
    count = get_k_anonymized_count(df_generalized, quasi_identifiers, k_val)
    remaining_counts.append(count)
    print(f"Für k={k_val}: {count} Datensätze ({count/total_records:.2%} erhalten)")

# Visualisierung des Trade-offs
plt.figure(figsize=(10, 6))
plt.plot(k_values, remaining_counts, marker='o', linestyle='-', color='red')
plt.title('Trade-off: k-Anonymität vs. verbleibende Datensätze')
plt.xlabel('Anonymitätsgrad k')
plt.ylabel('Anzahl der verbleibenden Datensätze')
plt.grid(True, linestyle='--', alpha=0.6)
plt.show()
